In [11]:
# simsiam
!experiment_name="debug";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=8 torchrun --nproc_per_node=1 --master_port=12345 train_multi_GPU.py \
    --wandb False --wandb_model run --sync_bn False --amp True --aux False \
    --model_name aspp_contrast_resnet50 --pre_trained deeplabv3_resnet50_coco.pth \
    --weight_only_backbone False \
    --data_path pascal-voc-2012 --num_classes 21 \
    --epochs 40 --batch_size 8 --batch_size_val 8 --memory_size 0 \
    --contrast 1 --sample self_pace3 \
    --loss_name aspp_loss --L1_loss 0 --L2_loss 0 --L3_loss 0\
    --name_date $name_date \
    2>&1 | tee $dir_log

Traceback (most recent call last):
  File "/home/zk/GIT/working/DCNet/fcn/train_multi_GPU.py", line 7, in <module>
    from train_utils import train_one_epoch, evaluate, create_lr_scheduler, init_distributed_mode, save_on_master, mkdir
  File "/home/zk/GIT/working/DCNet/fcn/train_utils/__init__.py", line 1, in <module>
    from .train_and_eval import train_one_epoch, evaluate, create_lr_scheduler
  File "/home/zk/GIT/working/DCNet/fcn/train_utils/train_and_eval.py", line 7, in <module>
    from train_utils.loss_manage import criterion
  File "/home/zk/GIT/working/DCNet/fcn/train_utils/loss_manage/__init__.py", line 6, in <module>
    from .aspp_loss import  ASPP_CONTRAST_Loss
  File "/home/zk/GIT/working/DCNet/fcn/train_utils/loss_manage/aspp_loss.py", line 3, in <module>
    from .samples import Sampling
  File "/home/zk/GIT/working/DCNet/fcn/train_utils/loss_manage/samples/__init__.py", line 2, in <module>
    from .sample_manage import Sampling
  File "/home/zk/GIT/working/DCNet/fcn